In [1]:
from plotnine import *
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import pickle
import seaborn as sns
import sys
from siuba import group_by, summarize, filter, mutate, arrange, spread, gather, _
from siuba.experimental.pivot import pivot_wider

sys.path.insert(0, "../")
from ope_estimators import *
from group_means import GroupMeanRegressor

%load_ext autoreload
%autoreload 2

In [2]:
with open("yeast.table", "r") as f:
    df = pd.read_table(f, header = None, delim_whitespace=True)

df.columns = ["Name", "X1", "X2", "X3", "X4", "X5", "X6", "X7", "X8", "Outcome"]

df = (df >> filter(_.Outcome != "ERL")
      >> filter(_.Outcome != "POX")
      >> filter(_.Outcome != "VAC")
      >> filter(_.Outcome != "EXC")
      >> filter(_.Outcome != "ME1")
      >> filter(_.Outcome != "ME2"))      
      

K = df["Outcome"].nunique()
n = len(df)

classes = df["Outcome"].unique()
class_labels = [i for i in range(K)]

label_map = {classes[i]:class_labels[i] for i in range(K)}


df["Y"] = df.apply(lambda r : label_map[r["Outcome"]], axis = 1)
df.head()

,Name,X1,X2,X3,X4,X5,X6,X7,X8,Outcome,Y
0,ADT1_YEAST,0.58,0.61,0.47,0.13,0.5,0.0,0.48,0.22,MIT,0
1,ADT2_YEAST,0.43,0.67,0.48,0.27,0.5,0.0,0.53,0.22,MIT,0
2,ADT3_YEAST,0.64,0.62,0.49,0.15,0.5,0.0,0.53,0.22,MIT,0
3,AAR2_YEAST,0.58,0.44,0.57,0.13,0.5,0.0,0.54,0.22,NUC,1
4,AATM_YEAST,0.42,0.44,0.48,0.54,0.5,0.0,0.48,0.22,MIT,0


In [3]:
df_discrete = df.copy() 

for col in ["X1", "X2", "X3", "X4", "X5", "X6", "X7", "X8"]:
    df_discrete[col] = np.where(df[col] > 0.5, 1, 0)


df_discrete.head()

,Name,X1,X2,X3,X4,X5,X6,X7,X8,Outcome,Y
0,ADT1_YEAST,1,1,0,0,0,0,0,0,MIT,0
1,ADT2_YEAST,0,1,0,0,0,0,1,0,MIT,0
2,ADT3_YEAST,1,1,0,0,0,0,1,0,MIT,0
3,AAR2_YEAST,1,0,1,0,0,0,1,0,NUC,1
4,AATM_YEAST,0,0,0,1,0,0,0,0,MIT,0


In [4]:
df_discrete.describe()

,X1,X2,X3,X4,X5,X6,X7,X8,Y
count,1299.000000,1299.000000,1299.000000,1299.000000,1299.000000,1299.000000,1299.000000,1299.000000,1299.000000
mean,0.377983,0.400308,0.575828,0.075443,0.006159,0.001540,0.524249,0.052348,1.419554
std,0.485070,0.490149,0.494407,0.264206,0.078265,0.039223,0.499604,0.222814,0.933185
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000
75%,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,2.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.000000


In [5]:
from sklearn.linear_model import LogisticRegression
from collections import Counter

def behavior_transformation(policy):
    new_policy = np.array([x if x > 0.05 else 0 for x in policy])
    new_policy = new_policy/sum(new_policy)

    return new_policy

def generate_ope_problem(n):
    model = LogisticRegression(penalty = "none", solver = "newton-cg")

    df_sample = df_discrete.sample(n, replace = True)
    
    X = df_sample[["X1", "X2", "X3", "X4", "X5", "X6", "X7", "X8"]].values
    Y = df_sample["Y"].values


    #Remove groups with only one observation
    X_cat = [str(j) for j in X]
    counts = Counter(X_cat)
    small_groups = [x for x in X_cat if counts[x] < 10]
    valid_indices = [j for j in range(X.shape[0]) if X_cat[j] not in small_groups]

    X = X[valid_indices,:]
    Y = Y[valid_indices]


    model.fit(X, Y)

    eval_probs = model.predict_proba(X)

    pi_e = {i:eval_probs[:,i] for i in range(K)}

    behavior_probs = np.apply_along_axis(behavior_transformation, 1, eval_probs)
    
    pi_b = {i:behavior_probs[:,i] for i in range (K)}
    
    A = np.array([np.random.choice(class_labels, p = behavior_probs[i,:]) for i in range(X.shape[0])])

    rewards = (A == Y).astype(int)

    return X, rewards, A, pi_b, pi_e 


In [6]:
from itertools import product
from sklearn.linear_model import LogisticRegression

N = 1000

logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S')

logger = logging.getLogger()


L = 1
n_grid = [500, 1000, 1500, 2000, 2500, 3000]

result_df = []
for n in n_grid:
    logger.info(f"Running n={n}")
    for trial in tqdm(range(N)):
        
        X, rewards, A, pi_b, pi_e = generate_ope_problem(n)
        model = GroupMeanRegressor("pred")
        
        Estimator = MultiActionOPEEstimator(X, rewards, A, pi_b, pi_e, L, class_labels, "LipImputeEstimator", model = model)
        lower, upper = Estimator.psi_hat()
        width = upper - lower 

        model = GroupMeanRegressor("upper")
        Estimator = MultiActionOPEEstimator(X, rewards, A, pi_b, pi_e, L, class_labels, "LipImputeEstimator", model = model, unsafe = True)
        _, eli_upper = Estimator.psi_hat() 

        model = GroupMeanRegressor("lower")
        Estimator = MultiActionOPEEstimator(X, rewards, A, pi_b, pi_e, L, class_labels, "LipImputeEstimator", model = model, unsafe = True)
        eli_lower, _ = Estimator.psi_hat()         
    
        eli_width = eli_upper - eli_lower


        result = {"trial":trial, "n":n, "width":width, "eli_width":eli_width}
        result_df.append(result)

result_df = pd.DataFrame.from_records(result_df)

2023-10-09 18:41:35 INFO     Running n=500


  0%|          | 0/1000 [00:00<?, ?it/s]

2023-10-09 18:47:36 INFO     Running n=1000


  0%|          | 0/1000 [00:00<?, ?it/s]

2023-10-09 19:33:45 INFO     Running n=1500


...

  0%|          | 0/1000 [00:00<?, ?it/s]

2023-10-09 20:39:01 INFO     Running n=2500


  0%|          | 0/1000 [00:00<?, ?it/s]

2023-10-09 21:33:58 INFO     Running n=3000


  0%|          | 0/1000 [00:00<?, ?it/s]

In [21]:
import pickle 

with open("results.pkl", "rb") as f:
    result_df = pickle.load(f)


agg_df = (result_df >> 
          group_by(_.n) >> 
          summarize(avg_width = 1000*_.width.mean(), avg_eli_width = 1000*_.eli_width.mean()) >>
          mutate(ratio = _.avg_eli_width/_.avg_width))

# agg_df = (result_df
#  .groupby(["n"])
#  .agg({"width":"mean", "eli_width":"mean"}))

agg_df

,n,avg_width,avg_eli_width,ratio
0,500,4.365816,6.565791,1.503909
1,1000,3.090582,4.886546,1.581109
2,1500,2.590696,3.893788,1.502989
3,2000,2.148935,3.180198,1.479895
4,2500,1.798020,2.637160,1.466702
5,3000,1.678935,2.405554,1.432786


In [22]:
agg_df.transpose().round(2).to_csv("eli_comparison.txt", sep="&")